# Datanews.io
import datanews
import pandas as pd
import math

# My API key. You can register a free API key and replace it with yours. 
datanews.api_key = '0e1oixqnn4f0fhdl3zod701ok'

In [ ]:
# A function to search and save to csv the original file, file removing rows with the same title, file removing rows with both the same 
# title and source:

def datanews_api_search(keyword):
    response = datanews.news (q=keyword,from_date="2018-01-01", to_date="2021-07-01",language=['en'],size=100,page=0)
    articles = response['hits']
    df_datanews = pd.DataFrame(articles)
    num_of_pages = int(math.ceil(response['numResults']/100))
    if num_of_pages>1:
        for i in range(1,num_of_pages):
            extra_page_response = datanews.news (q=keyword,from_date="2018-01-01", to_date="2021-08-01",language=['en'],size=100,page=i)
            extra_articles = extra_page_response['hits']
            df_datanews = df_datanews.append(extra_articles, ignore_index=True)                  
    else: pass
# store file in csv
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~+'''
# Removing punctuations in string
# Using loop + punctuation string
    for ele in keyword: 
        if ele in punc: 
            keyword = keyword.replace(ele, "") 
# remove duplicated rows by title:
    no_dup = df_datanews.drop_duplicates(subset=('title'),ignore_index=True)
# remove duplicated rows by title & source match: 
    no_dup_2 = df_datanews.drop_duplicates(subset=('title','source'),ignore_index=True)
# remove duplicated rows by title or source match: 
    no_dup_3 = no_dup.drop_duplicates(subset=('content'),ignore_index=True)
    df_datanews.to_csv(keyword+".csv")
    no_dup.to_csv(keyword+"_no_duplicated_title.csv")
    no_dup_2.to_csv(keyword+"_no_duplicated_title_source.csv")
    no_dup_3.to_csv(keyword+"_no_dup_title_content.csv")

# Search  different keywords: 
keyword1 = '"Food Access" + Jackson+ Mississippi'
datanews_api_search(keyword1)

keyword2 = '"Fertile Ground" + Mississippi'
datanews_api_search(keyword2)

keyword3 = '"Food Insecurity" + Jackson + Mississippi'
datanews_api_search(keyword3)

keyword4 = '"Food security" + Jackson + Mississippi'
datanews_api_search(keyword4)

keyword5 = '"Food desert" + Jackson + Mississippi'
datanews_api_search(keyword5)

keyword6 = '"Fertile Ground" + Jackson + Mississippi'
datanews_api_search(keyword6)

keyword7 = '"Fertile Ground Project" + Mississippi'
datanews_api_search(keyword7)

## Webscraping for full article: 
import requests
from bs4 import BeautifulSoup

# after combining and deduplicating files, a file with url was generated for BeautifulSoup
# import the file with the URL we need:
url_dataframe = pd.read_excel("For_BS.xlsx")

# Use a for loop to get the article content for each url and add back to the dataframe: 
allarticles = []
    
for i in range(len(url_dataframe)):
    url= url_dataframe.iloc[i,0]
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    articlebody = soup.find(class_='_61c55')
    # get text
    articletext = soup.find_all('p')[0:]
    thearticle = []
    paragraphtext = []    
    for paragraph in articletext[:-1]:
        # get the text only
        text = paragraph.get_text()
        paragraphtext.append(text)   
    thearticle.append(paragraphtext)
    myarticle = [' '.join(article) for article in thearticle]
    allarticles.append(myarticle)
    
# add a new column in the dataframe    
url_dataframe["Full Article"]=allarticles

# save to excel
url_dataframe.to_excel("All  with full article.xlsx")